Script sederhana scrapping data kos dari web Plantix menggunakan Selenium dan Python

Step 1 -
Import Library dan Modul yang dibutuhkan

In [1]:
import re, pandas as pd, time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait

Step 2 -
Konfigurasikan alamat web, webdriver dan service nya

In [2]:
executable_path = r"chromedriver-win64\chromedriver.exe"
service = Service(executable_path)
options = webdriver.ChromeOptions()
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_experimental_option("excludeSwitches", ["enable-automation"])   
options.add_experimental_option("useAutomationExtension", False) 
options.add_argument('--ignore-certificate-errors')
driver = webdriver.Chrome(service=service, options=options)

driver.get("https://plantix.net/en/library/plant-diseases/") 

Step 3 - filter dan scroll

In [3]:
start_time = time.time()

time.sleep(5)
pilFilter = driver.find_element(By. XPATH, '//*[@id="disease-filter"]/div/div[2]/div/div[2]/input')
pilFilter.click()
time.sleep(2)

try:
    kuki = WebDriverWait(driver, 3).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="cookie-consent-popup"]/div[2]/div/div[2]/div[2]/span/div/p'))
    )
    kuki.click()
    time.sleep(2)
except:
    print("Pop-up cookie tidak muncul atau tombol tidak ditemukan")

filJagung = driver.find_element(By.CSS_SELECTOR, "p.maize")
filJagung.click()
time.sleep(2)

last_height = driver.execute_script("return document.body.scrollHeight")

while True:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)
    
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

list_df = []

time.sleep(3)

In [4]:
cards = driver.find_elements(By.CSS_SELECTOR, "a.disease-content")

links = []

for card in cards:
    href = card.get_attribute("href")
    if href:
        links.append(href)

print("Total:", len(links))
links[:10]

Total: 111


['https://plantix.net/en/library/plant-diseases/600373/flower-chafer/',
 'https://plantix.net/en/library/plant-diseases/700049/boron-deficiency/',
 'https://plantix.net/en/library/plant-diseases/999999/healthy/',
 'https://plantix.net/en/library/plant-diseases/800107/leaf-variegation/',
 'https://plantix.net/en/library/plant-diseases/200046/maize-chlorotic-mottle-virus/',
 'https://plantix.net/en/library/plant-diseases/600322/bagrada-bug/',
 'https://plantix.net/en/library/plant-diseases/900013/herbicide-bleach/',
 'https://plantix.net/en/library/plant-diseases/600030/spotted-stemborer/',
 'https://plantix.net/en/library/plant-diseases/600088/cucumber-beetle/',
 'https://plantix.net/en/library/plant-diseases/700007/iron-deficiency/']

Step 4
Ambil data yang dibutuhkan

In [5]:
def scrape_detail(url):
    driver.get(url)
    time.sleep(2)

    try:
        nama = driver.find_element(By.CLASS_NAME, "disease-name").text.strip()
    except:
        try:
            nama = driver.find_element(By.TAG_NAME, "h1").text.strip()
        except:
            nama = ""

    try:
        nama_ilmiah = driver.find_element(By.CLASS_NAME, "scientific-name").text.strip()
    except:
        try:
            nama_ilmiah = driver.find_element(By.CSS_SELECTOR, "h1 + h2").text.strip()
        except:
            nama_ilmiah = ""

    try:
        jenis = driver.find_element(By.CLASS_NAME, "class_text").text.strip()
    except:
        try:
            jenis = driver.find_element(By.CSS_SELECTOR, "p.disease-type").text.strip()
        except:
            jenis = ""

    try:
        gejala = driver.find_element(By.CSS_SELECTOR, '[data-cy="symptoms-card"] p').text.strip()
    except:
        gejala = ""

    rekomendasi_list = []

    try:
        orgs = driver.find_elements(By.CSS_SELECTOR, '[data-cy="biological-control-card"] p')
        rekomendasi_list += [o.text.strip() for o in orgs if o.text.strip()]
    except:
        pass

    try:
        chems = driver.find_elements(By.CSS_SELECTOR, '[data-cy="chemical-control-card"] p')
        rekomendasi_list += [c.text.strip() for c in chems if c.text.strip()]
    except:
        pass

    rekomendasi = " ".join(rekomendasi_list)

    try:
        penyebab = driver.find_element(By.CSS_SELECTOR, '[data-cy="trigger-card"] p').text.strip()
    except:
        penyebab = ""

    try:
        lis = driver.find_elements(By.CSS_SELECTOR, '[data-cy="preventive-measures-card"] li')
        pencegahan = "; ".join(li.text.strip() for li in lis if li.text.strip())
    except:
        pencegahan = ""

    return {
        # "url": url,
        "nama": nama,
        "namaIlmiah": nama_ilmiah,
        "jenis": jenis,
        "gejala": gejala,
        "rekomendasi": rekomendasi,
        "penyebab": penyebab,
        "pencegahan": pencegahan
    }

In [6]:
hasil = []

print("Testing one link only:")
print(links[0])

try:
    data = scrape_detail(links[0])
    hasil.append(data)
except Exception as e:
    print("Error:", e)

hasil

Testing one link only:
https://plantix.net/en/library/plant-diseases/600373/flower-chafer/


[{'nama': 'Flower Chafer',
  'namaIlmiah': 'Oxycetonia versicolor',
  'jenis': 'Insect',
  'gejala': 'The pest can cause significant yield loss due to damage to reproductive organs. Adult beetles devour the flowers and buds by feeding on pollen, anthers and other reproductive parts inside the flowers. In cotton, they also attack tender bolls. They have also been observed chewing the tender shoots of eggplant crops and other tender tissue of their hosts, especially in the immature phase.',
  'rekomendasi': 'There are no known biological treatments at the moment. Always consider an integrated approach with preventive measures along with biological treatments, if available. There are no known chemical treatments at the moment. Always consider an integrated approach with preventive measures along with biological treatments, if available. There are no known chemical treatments at the moment.',
  'penyebab': 'Damage is caused by the chafer adults. Flower chafers are day-flying beetles and ma

In [7]:
hasil = []

for i, url in enumerate(links):
    print(f"[{i+1}/{len(links)}] Scraping: {url}")

    try:
        data = scrape_detail(url)
        hasil.append(data)
    except Exception as e:
        print("Error:", url, e)

    time.sleep(1)

[1/111] Scraping: https://plantix.net/en/library/plant-diseases/600373/flower-chafer/
[2/111] Scraping: https://plantix.net/en/library/plant-diseases/700049/boron-deficiency/
[3/111] Scraping: https://plantix.net/en/library/plant-diseases/999999/healthy/
[4/111] Scraping: https://plantix.net/en/library/plant-diseases/800107/leaf-variegation/
[5/111] Scraping: https://plantix.net/en/library/plant-diseases/200046/maize-chlorotic-mottle-virus/
[6/111] Scraping: https://plantix.net/en/library/plant-diseases/600322/bagrada-bug/
[7/111] Scraping: https://plantix.net/en/library/plant-diseases/900013/herbicide-bleach/
[8/111] Scraping: https://plantix.net/en/library/plant-diseases/600030/spotted-stemborer/
[9/111] Scraping: https://plantix.net/en/library/plant-diseases/600088/cucumber-beetle/
[10/111] Scraping: https://plantix.net/en/library/plant-diseases/700007/iron-deficiency/
[11/111] Scraping: https://plantix.net/en/library/plant-diseases/900005/pesticide-damage/
[12/111] Scraping: https:

In [8]:
import json

with open("plantix.json", "w", encoding="utf-8") as f:
    json.dump(hasil, f, ensure_ascii=False, indent=2)

In [ ]:
df = pd.DataFrame(hasil)
df.to_csv("plantix.csv")
df.head()

,nama,namaIlmiah,jenis,gejala,rekomendasi,penyebab,pencegahan
0,Flower Chafer,Oxycetonia versicolor,Insect,The pest can cause significant yield loss due ...,There are no known biological treatments at th...,Damage is caused by the chafer adults. Flower ...,The beetle migrates between crops and has a wi...
1,Boron Deficiency,Boron Deficiency,Deficiency,"Symptoms are variable, depending on the crop a...",Make sure to have healthy soils with good orga...,Boron deficiency is usually observed in soils ...,Avoid soils with a high pH and are rich in cla...
2,Healthy,Healthy,Other,Fertilize with the right fertilizer mixture an...,. . .,.,.
3,Leaf Variegation,Chimera,Other,Leaf variegation shows as uneven white to yell...,As there is no direct environmental cause for ...,Leaf variegation is a genetic or physiologic a...,Use certified seeds from known varieties/hybri...
4,Maize Chlorotic Mottle Virus,MCMV,Virus,Symptoms vary in severity depending on the dif...,There is no direct control of diseases caused ...,The symptoms are caused by a virus (MCMV) that...,"Plant resistant varieties, if available, as it..."
